Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# Experiment Management with ClearML

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/experiment_management/unet_segmentation_3d_ignite_clearml.ipynb)

This tutorial shows how to use ClearML to manage MONAI experiments. You can integrate ClearML into your code using MONAI's built-in handlers: `ClearMLImageHandler`, `ClearMLStatsHandler`, and `ModelCheckpoint`.

The MONAI example used here is [3D segmentation with UNet](https://github.com/Project-MONAI/tutorials/blob/main/3d_segmentation/unet_segmentation_3d_ignite.ipynb).

## Setup environment

`clearml` comes as part of the `monai[all]` installation. For more information about integrating ClearML into your MONAI code, see [here](https://clear.ml/docs/latest/docs/integrations/monai). For more information about using ClearML (experiment management, data management, pipelines, model serving, and more), see [ClearML's official documentation](https://clear.ml/docs/latest/docs/).

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[ignite, nibabel, tensorboard, clearml]"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.5/266.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.6 MB/s eta 0:00:00


## Setup imports

In [2]:
import glob
import logging
import os
import shutil
import sys
import tempfile

import nibabel as nib
import numpy as np
from monai.config import print_config
from monai.data import ArrayDataset, create_test_image_3d, decollate_batch, DataLoader
from monai.handlers import (
    MeanDice,
    StatsHandler,
)

# import the clearml handlers
from monai.handlers.clearml_handlers import ClearMLImageHandler, ClearMLStatsHandler
from monai.losses import DiceLoss
from monai.networks.nets import UNet
from monai.transforms import (
    Activations,
    EnsureChannelFirst,
    AsDiscrete,
    Compose,
    LoadImage,
    RandSpatialCrop,
    Resize,
    ScaleIntensity,
)
from monai.utils import first

import ignite
import torch

print_config()

MONAI version: 1.5.dev2444
Numpy version: 1.26.4
Pytorch version: 2.5.0+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c70fbd8ff919cabaacfabcbdbf28aa435ae622f9
MONAI __file__: /usr/local/lib/python3.10/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.13.1
Pillow version: 10.4.0
Tensorboard version: 2.17.0
gdown version: 5.2.0
TorchVision version: 0.20.0+cu121
tqdm version: 4.66.6
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.44.2
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: 1.16.5

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommend

## Setup ClearML ⚓

1. To keep track of your experiments and/or data, ClearML needs to communicate to a server. You have 2 server options:

  * Sign up for free to the [ClearML Hosted Service](https://app.clear.ml/)
  * Set up your own server, see [here](https://clear.ml/docs/latest/docs/deploying_clearml/clearml_server).

2. Add your ClearML credentials below. ClearML will use these credentials to connect to your server (see instructions for generating credentials [here](https://clear.ml/docs/latest/docs/getting_started/ds/ds_first_steps/#jupyter-notebook)).

In [ ]:
# clearml credentials
%env CLEARML_WEB_HOST=''
%env CLEARML_API_HOST=''
%env CLEARML_FILES_HOST=''
%env CLEARML_API_ACCESS_KEY=''
%env CLEARML_API_SECRET_KEY=''

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [4]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/tmp/tmpalv7toxp


## Setup logging

In [5]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Setup demo data

In [6]:
for i in range(40):
    im, seg = create_test_image_3d(128, 128, 128, num_seg_classes=1)

    n = nib.Nifti1Image(im, np.eye(4))
    nib.save(n, os.path.join(root_dir, f"im{i}.nii.gz"))

    n = nib.Nifti1Image(seg, np.eye(4))
    nib.save(n, os.path.join(root_dir, f"seg{i}.nii.gz"))

images = sorted(glob.glob(os.path.join(root_dir, "im*.nii.gz")))
segs = sorted(glob.glob(os.path.join(root_dir, "seg*.nii.gz")))

## Setup transforms, dataset

In [7]:
# Define transforms for image and segmentation
imtrans = Compose(
    [
        LoadImage(image_only=True),
        ScaleIntensity(),
        EnsureChannelFirst(),
        RandSpatialCrop((96, 96, 96), random_size=False),
    ]
)
segtrans = Compose(
    [
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        RandSpatialCrop((96, 96, 96), random_size=False),
    ]
)

# Define nifti dataset, dataloader
ds = ArrayDataset(images, imtrans, segs, segtrans)
# loader = DataLoader(ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available())
loader = DataLoader(ds, batch_size=10, num_workers=2, pin_memory=False)
im, seg = first(loader)
print(im.shape, seg.shape)

torch.Size([10, 1, 96, 96, 96]) torch.Size([10, 1, 96, 96, 96])


## Create Model, Loss, Optimizer

In [8]:
# Create UNet, DiceLoss and Adam optimizer

device = None  # torch.device("cuda:0")
net = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)

loss = DiceLoss(sigmoid=True)
lr = 1e-3
opt = torch.optim.Adam(net.parameters(), lr)

## Create supervised_trainer using ignite

In [9]:
# Create trainer
trainer = ignite.engine.create_supervised_trainer(net, opt, loss, device, False)

## Set up event handlers for checkpointing and logging

Using a ClearML handler creates a ClearML Task, which captures your experiment's models, scalars, images, and more.

The console output displays the task ID and a link to the task's page in the ClearML web UI.

In [10]:
# optional section for checkpoint and clearml logging
# adding checkpoint handler to save models (network
# params and optimizer stats) during training
log_dir = os.path.join(root_dir, "logs")
checkpoint_handler = ignite.handlers.ModelCheckpoint(log_dir, "net", n_saved=10, require_empty=False)
trainer.add_event_handler(
    event_name=ignite.engine.Events.EPOCH_COMPLETED,
    handler=checkpoint_handler,
    to_save={"net": net, "opt": opt},
)

# StatsHandler prints loss at every iteration
# user can also customize print functions and can use output_transform to convert
# engine.state.output if it's not a loss value
train_stats_handler = StatsHandler(name="trainer", output_transform=lambda x: x)
train_stats_handler.attach(trainer)


# ClearMLStatsHandler plots loss at every iteration
# Creates a ClearML Task which logs the scalar plots
task_name = "UNet segmentation 3d"
project_name = "MONAI example"

train_clearml_stats_handler = ClearMLStatsHandler(
    task_name=task_name, project_name=project_name, log_dir=log_dir, output_transform=lambda x: x
)
train_clearml_stats_handler.attach(trainer)

ClearML Task: created new task id=c9e03a740b4c442eb7421dc75077af50
ClearML results page: https://app.clear.ml/projects/35c4574a6ec1425cba6cfb1060765bb3/experiments/c9e03a740b4c442eb7421dc75077af50/output/log
2024-11-03 10:30:44,070 - clearml.Task - INFO - Storing jupyter notebook directly as code


## Add Validation every N epochs

In [12]:
# optional section for model validation during training
validation_every_n_epochs = 1
# Set parameters for validation
metric_name = "Mean_Dice"
# add evaluation metric to the evaluator engine
val_metrics = {metric_name: MeanDice()}
post_pred = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
post_label = Compose([AsDiscrete(threshold=0.5)])
# Ignite evaluator expects batch=(img, seg) and
# returns output=(y_pred, y) at every iteration,
# user can add output_transform to return other values
evaluator = ignite.engine.create_supervised_evaluator(
    net,
    val_metrics,
    device,
    True,
    output_transform=lambda x, y, y_pred: (
        [post_pred(i) for i in decollate_batch(y_pred)],
        [post_label(i) for i in decollate_batch(y)],
    ),
)

# create a validation data loader
val_imtrans = Compose(
    [
        LoadImage(image_only=True),
        ScaleIntensity(),
        EnsureChannelFirst(),
        Resize((96, 96, 96)),
    ]
)
val_segtrans = Compose(
    [
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        Resize((96, 96, 96)),
    ]
)
val_ds = ArrayDataset(images[21:], val_imtrans, segs[21:], val_segtrans)
# val_loader = DataLoader(val_ds, batch_size=5, num_workers=8, pin_memory=torch.cuda.is_available())
val_loader = DataLoader(val_ds, batch_size=5, num_workers=2, pin_memory=False)


@trainer.on(ignite.engine.Events.EPOCH_COMPLETED(every=validation_every_n_epochs))
def run_validation(engine):
    evaluator.run(val_loader)


# Add stats event handler to print validation stats via evaluator
val_stats_handler = StatsHandler(
    name="evaluator",
    # no need to print loss value, so disable per iteration output
    output_transform=lambda x: None,
    # fetch global epoch number from trainer
    global_epoch_transform=lambda x: trainer.state.epoch,
)
val_stats_handler.attach(evaluator)

# add handler to record metrics to clearml at every validation epoch
val_clearml_stats_handler = ClearMLStatsHandler(
    log_dir=log_dir,
    # no need to plot loss value, so disable per iteration output
    output_transform=lambda x: None,
    # fetch global epoch number from trainer
    global_epoch_transform=lambda x: trainer.state.epoch,
)
val_clearml_stats_handler.attach(evaluator)

# add handler to draw the first image and the corresponding
# label and model output in the last batch
# here we draw the 3D output as GIF format along Depth
# axis, at every validation epoch
val_clearml_image_handler = ClearMLImageHandler(
    task_name=task_name,
    project_name=project_name,
    log_dir=log_dir,
    batch_transform=lambda batch: (batch[0], batch[1]),
    output_transform=lambda output: output[0],
    global_iter_transform=lambda x: trainer.state.epoch,
)
evaluator.add_event_handler(
    event_name=ignite.engine.Events.EPOCH_COMPLETED,
    handler=val_clearml_image_handler,
)

## Run training loop

In [13]:
# create a training data loader
train_ds = ArrayDataset(images[:20], imtrans, segs[:20], segtrans)
train_loader = DataLoader(
    train_ds,
    batch_size=5,
    shuffle=True,
    num_workers=2,
    # pin_memory=torch.cuda.is_available(),
    pin_memory=False,
)

max_epochs = 10
trainer.run(train_loader, max_epochs)

2024-11-03 10:33:38,995 - INFO - Epoch: 1/10, Iter: 1/4 -- Loss: 0.6257 
2024-11-03 10:33:44,424 - INFO - Epoch: 1/10, Iter: 2/4 -- Loss: 0.6480 
2024-11-03 10:33:51,319 - INFO - Epoch: 1/10, Iter: 3/4 -- Loss: 0.6237 
2024-11-03 10:33:59,059 - INFO - Epoch: 1/10, Iter: 4/4 -- Loss: 0.5821 
2024-11-03 10:34:02,483 - clearml.storage - INFO - Uploading: 55.07MB to /tmp/.clearml.upload_model_vad4dg8k.tmp


██████████████████████████████▉ 100% | 55.07/55.07 MB [00:01<00:00, 32.06MB/s]: 

2024-11-03 10:34:04,228 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/MONAI%20example/UNet%20segmentation%203d.c9e03a740b4c442eb7421dc75077af50/models/tmpqeno0tgh


2024-11-03 10:34:11,999 - INFO - Epoch[1] Metrics -- Mean_Dice: 0.2175 
2024-11-03 10:34:22,828 - INFO - Epoch[1] Metrics -- Mean_Dice: 0.2175 
2024-11-03 10:34:30,584 - INFO - Epoch: 2/10, Iter: 1/4 -- Loss: 0.6001 
2024-11-03 10:34:38,635 - INFO - Epoch: 2/10, Iter: 2/4 -- Loss: 0.5744 
2024-11-03 10:34:45,662 - INFO - Epoch: 2/10, Iter: 3/4 -- Loss: 0.6105 
2024-11-03 10:34:51,861 - INFO - Epoch: 2/10, Iter: 4/4 -- Loss: 0.5510 
2024-11-03 10:34:54,764 - clearml.storage - INFO - Uploading: 55.07MB to /tmp/.clearml.upload_model__n8pbaer.tmp


██████████████████████████████▉ 100% | 55.07/55.07 MB [00:01<00:00, 34.02MB/s]: 

...

2024-11-03 10:41:33,174 - INFO - Epoch[9] Metrics -- Mean_Dice: 0.3370 
2024-11-03 10:41:45,329 - INFO - Epoch[9] Metrics -- Mean_Dice: 0.3370 
2024-11-03 10:41:53,656 - INFO - Epoch: 10/10, Iter: 1/4 -- Loss: 0.4579 
2024-11-03 10:42:01,387 - INFO - Epoch: 10/10, Iter: 2/4 -- Loss: 0.4487 
2024-11-03 10:42:06,797 - INFO - Epoch: 10/10, Iter: 3/4 -- Loss: 0.4301 
2024-11-03 10:42:12,571 - INFO - Epoch: 10/10, Iter: 4/4 -- Loss: 0.4469 
2024-11-03 10:42:16,117 - clearml.storage - INFO - Uploading: 55.07MB to /tmp/.clearml.upload_model_xmty8i5q.tmp


██████████████████████████████▉ 100% | 55.07/55.07 MB [00:01<00:00, 30.86MB/s]: 

2024-11-03 10:42:17,927 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/MONAI%20example/UNet%20segmentation%203d.c9e03a740b4c442eb7421dc75077af50/models/tmpc8kgr25v


2024-11-03 10:42:24,255 - INFO - Epoch[10] Metrics -- Mean_Dice: 0.4036 
2024-11-03 10:42:35,671 - INFO - Epoch[10] Metrics -- Mean_Dice: 0.4036 


State:
	iteration: 40
	epoch: 10
	epoch_length: 4
	max_epochs: 10
	output: 0.44685739278793335
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'monai.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

## Visualize results

ClearML captures the models, scalar plots, and images logged with `ModelCheckpoint`, `ClearMLImageHandler`, and `ClearMLStatsHandler` respectively. View them in ClearML's web UI. When a task is created, the console output displays the task ID and a link to the task's page in the ClearML web UI.

### Models
All model checkpoints logged with ModelCheckpoint can be viewed in the task's **Artifacts** tab.





![MONAI ClearML Models](https://github.com/Project-MONAI/tutorials/blob/main/figures/monai_clearml_models.png?raw=1)

### Scalars

View the logged metric plots in the task's **Scalars** tab.



![MONAI ClearML scalars.png](https://github.com/Project-MONAI/tutorials/blob/main/figures/monai_clearml_scalars.png?raw=1)

### Debug Samples

View all images logged through ClearMLImageHandler in the task's **Debug Samples** tab. You can view the samples by metric at any iteration.

![MONAI ClearML Debug Samples.png](https://github.com/Project-MONAI/tutorials/blob/main/figures/monai_clearml_debug_samples.png?raw=1)

## Cleanup data directory

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)


## Close the ClearML Task
This changes the task status to `Completed`.

In [ ]:
val_clearml_image_handler.clearml_task.close()